In [1]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
from Classes.cognitive_model_agents import MODELS, COMPLEX_PARAMETER_MODELS
from Classes.cognitive_model_agents import *
from Classes.parameter_recovery import ParameterFit

In [3]:
# Create paths for data and results
data_folder = Path('..', 'data', 'human')
folder_2P = Path('..', 'reports', 'MLE')
folder_2P.mkdir(parents=True, exist_ok=True)
best_fit_file_2P = Path(folder_2P, f'best_fit_2P.json')

folder_all = Path('..', 'reports', 'MLE')
folder_all.mkdir(parents=True, exist_ok=True)
best_fit_file_all = Path(folder_all, f'best_fit_all.json')

## Fit to 2-player data

In [4]:
#Load 2P data

file_name = '2-player-UR.csv'
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data.head()

Loading data from ..\data\human\2-player-UR.csv...


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,source,treatment,group,round,player,choice,score,attendance,ac_score,threshold,num_players,room,decision
0,0,0,0,0,0,human,score + grid,Grupo-0001,1,405186428721292,0,0,1,25,0.5,2,Grupo-0001,0
1,1,1,1,1,1,human,score + grid,Grupo-0001,1,728608683559397,1,1,1,25,0.5,2,Grupo-0001,1
2,2,2,2,2,2,human,score + grid,Grupo-0001,2,405186428721292,1,1,1,25,0.5,2,Grupo-0001,1
3,3,3,3,3,3,human,score + grid,Grupo-0001,2,728608683559397,0,0,1,25,0.5,2,Grupo-0001,0
4,4,4,4,4,4,human,score + grid,Grupo-0001,3,405186428721292,0,0,1,25,0.5,2,Grupo-0001,0


In [5]:
# Create optimization hyperparameters
hyperparameters = {
    'init_points':1,
    'n_iter':1
}

#Initialize records
best_fit = {
    'model_name': list(),
    'deviance': list(),
    'AIC': list(),
    'free_parameters': list()
}

In [6]:
best_fit = dict()
for model in tqdm(MODELS, desc='Models '):
    if model in COMPLEX_PARAMETER_MODELS:        
    	continue
    print(f'Fitting data to model {model.name()}...')
    best_fit['model_name'] = model.name()
    print('Creating parameter recovery class...')
    pf = ParameterFit(
        agent_class=model,
        model_name=model.name(),
        data=data,
        optimizer_name='bayesian'
    )
    print('Running bayesian optimizer...')
    res = pf.get_optimal_parameters(hyperparameters)
    best_fit.update(res)
print(best_fit)

Models :   0%|          | 0/21 [00:00<?, ?it/s]

Fitting data to model WSLS-M1...
Creating parameter recovery class...
Running bayesian optimizer...
Finding deviance for 2 players and threshold 0.5...
|   iter    |  target   | heuris... | invers... | wsls_s... |
-------------------------------------------------------------
| 1         | -3.182e+0 | 0.417     | 46.38     | 0.001144  |
| 2         | -1.413e+0 | 0.8504    | 15.94     | 9.555     |
Optimal parameters for 2 players and threshold 0.5:
{'heuristic_strength': 0.417022004702574, 'inverse_temperature': 46.38044308685596, 'wsls_strength': 0.0011437481734488664}
Deviance: -3182.472124704105
AIC: 6370.94424940821
--------------------------------------------------
Fitting data to model WSLS-M2...
Creating parameter recovery class...
Running bayesian optimizer...
Finding deviance for 2 players and threshold 0.5...
|   iter    |  target   | heuris... | invers... | wsls_s... |
-------------------------------------------------------------
| 1         | -3.155e+0 | 0.417     | 46.38   

# MODELS is a dictionary with the free parameters for 
# each model and comes from Classes.cognitive_model_agents
for model_name, model in tqdm(MODELS.items(), desc='Models '):
    # if model_name not in ['MFP-M3']:
    # 	continue
    print(f'Fitting data to model {model_name}...')
    best_fit['model_name'].append(model_name)
    print('Creating parameter recovery class...')
    pf = ParameterFit(
        agent_class=model["class"],
        model_name=model_name,
        free_parameters=model["free_parameters"],
        data=data,
        optimizer_name='bayesian'
    )
    print('Running bayesian optimizer...')
    res = pf.get_optimal_parameters(hyperparameters)
    best_fit['deviance'].append(-res["target"])
    best_fit['AIC'].append(2*len(model["free_parameters"]) - 2*res["target"])
    best_fit['free_parameters'].append(res["params"])
print(best_fit)

In [ ]:
list_lens = [len(val) for key, val in best_fit.items()]
max_len = max(list_lens)
assert(np.all(x == max_len for x in list_lens))
with open(best_fit_file_2P, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_2P}')

## Fit to all player data

In [ ]:
# Load data into a dataframe
file_names = [
	# '2-player-UR.csv',
	'3-player-IU.csv',
	'4-player-IU.csv',
	'5-player-IU.csv',
	'6-player-IU.csv',
	'7-player-IU.csv',
	'8-player-IU.csv',
	'9-player-IU.csv',
	'11-player-IU.csv',
	'12-player-IU.csv',
]
df_list = list()
for file_name in file_names:
	file = data_folder / Path(file_name)
	print(f'Loading data from {file}...')
	df = pd.read_csv(file)
	df_list.append(df)
data = pd.concat(df_list, ignore_index=True)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head()

In [ ]:
# Create optimization hyperparameters
hyperparameters = {
    'init_points':4,
    'n_iter':8
}

#Initialize records
best_fit = {
    'model_name': list(),
    'deviance': list(),
    'num_parameters': list(),
    'AIC': list(),
    'free_parameters': list()
}

# MODELS is a dictionary with the free parameters for 
# each model and comes from Classes.cognitive_model_agents
for model_name, model in tqdm(MODELS.items(), desc='Models '):
    # if model_name not in ['QAttendance', 'QFairness']:
    # 	continue
    print(f'Fitting data to model {model_name}...')
    best_fit['model_name'].append(model_name)
    print('Creating parameter recovery class...')
    pf = ParameterFit(
        agent_class=model["class"],
        model_name=model_name,
        free_parameters=model["free_parameters"],
        data=data,
        optimizer_name='bayesian',
        with_treatment=False
    )
    print('Running bayesian optimizer...')
    res = pf.get_optimal_parameters(hyperparameters)
    best_fit['deviance'].append(-res["target"])
    best_fit['num_parameters'].append(len(model["free_parameters"]))
    best_fit['AIC'].append(2*len(model["free_parameters"]) - 2*res["target"])
    best_fit['free_parameters'].append(res["params"])
print(best_fit)
with open(best_fit_file_all, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_all}')

In [ ]:
list_lens = [len(val) for key, val in best_fit.items()]
max_len = max(list_lens)
assert(np.all(x == max_len for x in list_lens))
with open(best_fit_file_all, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_all}')